<a href="https://colab.research.google.com/github/pedroafr/notebooks/blob/master/Appsflyer_x_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Analytics + AppsFlyer Attribution Study**

We are going to analyze the conversion attribution for Android devices from 1st September to 30th September.

**Providing the credentials to the runtime**

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Enabling Data Table Display

In [0]:
%load_ext google.colab.data_table

**JOIN Appsflyer conversions with GA Transactions**

*   Each conversion event on Appsflyer can match N transactions on Google Analytics
*   Every row on Google Analytics is unique due to transactionId (policy number)


In [0]:
%%bigquery --project youse-bigquery-dw
WITH 
  appsflyer AS (
    SELECT
      Customer_User_Id AS hashed_email
      , Event_Time AS conversion_time
      , CONCAT ("app / ", Media_Source) AS source_medium
      , Event_Revenue AS revenue
      , JSON_EXTRACT_SCALAR(Event_Value,"$.cross_sell") AS cross_sell
      , JSON_EXTRACT_SCALAR(Event_Value,"$.af_content_type") AS product
    FROM `youse-bigquery-dw.tests.appsflyer_test1`
  ),

  google_analytics_transactions AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , DATETIME(TIMESTAMP_MILLIS(visitStartTime*1000 + hits.time), "America/Sao_Paulo") AS conversion_time
      , CONCAT(trafficSource.source, " / ", trafficSource.medium) AS source_medium
      , (hits.transaction.transactionRevenue / 1e6) AS revenue
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 16) AS cross_sell
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 1) AS sales_channel
      , hits.transaction.transactionId AS transaction_id
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "TRANSACTION"
  ),

  google_analytics_items AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , hits.transaction.transactionId AS transaction_id
      , hits.item.productName AS product
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "ITEM"
  ),

  google_analytics AS (
    SELECT
      ga_transactions.hashed_email
      , ga_transactions.conversion_time
      , ga_transactions.source_medium
      , ga_transactions.revenue
      , ga_transactions.cross_sell
      , ga_items.product
      , ga_transactions.sales_channel
      , ga_transactions.transaction_id
    FROM
      google_analytics_transactions ga_transactions
    LEFT JOIN
      google_analytics_items ga_items
    ON 
      ga_transactions.hashed_email = ga_items.hashed_email
      AND ga_transactions.transaction_id = ga_items.transaction_id
  ),

  appsflyer_google_analytics AS (
    SELECT 
      af.hashed_email AS af_hashed_email
      , af.conversion_time AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , af.source_medium AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , af.revenue AS af_revenue
      , ga.revenue AS ga_revenue
      , af.cross_sell AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , af.product AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
      , ROW_NUMBER() OVER (
          PARTITION BY 
            af.hashed_email
            , af.conversion_time 
          ORDER BY 
            ABS(DATETIME_DIFF(ga.conversion_time, DATETIME(af.conversion_time),SECOND))) AS row_number
    FROM
      appsflyer af
    LEFT JOIN google_analytics ga
    ON 
      af.hashed_email = ga.hashed_email
      AND af.revenue = ga.revenue
    ORDER BY
      af.hashed_email 
  )

SELECT 
  *
FROM 
  appsflyer_google_analytics
ORDER BY hashed_email

,af_hashed_email,af_conversion_time,ga_conversion_time,af_source_medium,ga_source_medium,af_revenue,ga_revenue,af_cross_sell,ga_cross_sell,ga_transaction_id,af_product,ga_product,ga_sales_channel,row_number
0,0051600b756a99f40b451aced72c9290998d4750cdd3a9...,2019-09-26 21:30:05+00:00,2019-09-26 21:32:08.046,app / muvmobile_int,google / cpc,140.14,140.14,false,false,5003110344856,Seguro Auto,Seguro Auto,Digital,1
1,00677ba7557c556d4cd015dc064329e631c4bcfc0b4e03...,2019-09-06 23:03:53+00:00,2019-09-06 23:03:56.639,app / googleadwords_int,app / app_youse,179.38,179.38,false,false,5003110337342,Seguro Auto,Seguro Auto,Digital,1
2,00e37cb4c615ca912d547279407f4cab0c640d0de442c5...,2019-09-11 20:15:58+00:00,2019-09-11 20:16:03.122,app / doubleclick_int,app / app_youse,169.31,169.31,false,false,5003110338961,Seguro Auto,Seguro Auto,Digital,1
3,0120036244c1d5f9ccb335798e9fd275cc36798aa6bed4...,2019-09-25 17:27:53+00:00,2019-09-25 17:04:57.000,app / googleadwords_int,202616.apps.zdusercontent.com / referral,10.63,10.63,false,false,5001410102477,Seguro Residencial,Seguro Residencial,Call-Center,2
4,0120036244c1d5f9ccb335798e9fd275cc36798aa6bed4...,2019-09-25 17:27:53+00:00,2019-09-25 17:27:58.110,app / googleadwords_int,202616.apps.zdusercontent.com / referral,10.63,10.63,false,false,5001410102479,Seguro Residencial,Seguro Residencial,Call-Center,1
5,01260d5174081fe48a54c63c18ce21ebddbcbcce57c107...,2019-09-16 16:19:10+00:00,2019-09-16 16:19:13.977,app / googleadwords_int,app / app_youse,192.05,192.05,false,false,5003110340802,Seguro Auto,Seguro Auto,Digital,1
6,0126a19cc233dc07789cce733cc8a6b8ae2ef452d5eb8e...,2019-09-17 15:19:06+00:00,2019-09-17 15:19:11.323,app / googleadwords_int,app / app_youse,69.92,69.92,false,false,5009110073789,Seguro Vida,Seguro Vida,Digital,1
7,012805e83c2233addde70c94d6b0b7cdd0d95906eb5aed...,2019-09-05 20:16:09+00:00,2019-09-05 20:16:12.177,app / googleadwords_int,202616.apps.zdusercontent.com / referral,226.81,226.81,false,false,5003110336827,Seguro Auto,Seguro Auto,Call-Center,1
8,018ae3d537666068939c65f3c57e7d87e2ba09d0726d41...,2019-09-27 19:13:10+00:00,2019-09-27 19:13:15.403,app / googleadwords_int,202616.apps.zdusercontent.com / referral,100.21,100.21,false,false,5003110345258,Seguro Auto,Seguro Auto,Call-Center,1
9,02375af5544a31171a57aebdf2e8d5a9cfac91727609ad...,2019-09-12 09:45:53+00:00,2019-09-12 09:45:59.205,app / googleadwords_int,google / cpc,54.02,54.02,false,false,5003110339130,Seguro Auto,Seguro Auto,Digital,1


The query result shown above returned more rows than the number of conversion events on appsflyer table and it happens because of the reasons below:

1.   We don't have the policy number on appsflyer conversion events
2.   There may be transactions on Google Analytics not sent to appsflyer according to the identification rules set in the worker (backend - rules to be investigated), so there will be matches with google analytics transactions not sent to appsflyer (like some cross selling transactions)

**AppsFlyer Deduplication**

We are going to deduplicate and try to get just the correct Google Analytics transaction (as we don't have the policy number on appsflyer we are going to assume that the shorter is the time difference between the appsflyer conversion event and the google analytics transaction, the greater the chance that it will be the same transaction in both tools).

In the code below we are going to add the necessary row_number to help us deduplicate and get the rows with row_number 1 (the shorter time difference between conversion event and transaction in GA)

In [0]:
%%bigquery --project youse-bigquery-dw
WITH 
  appsflyer AS (
    SELECT
      Customer_User_Id AS hashed_email
      , Event_Time AS conversion_time
      , CONCAT ("app / ", Media_Source) AS source_medium
      , Event_Revenue AS revenue
      , JSON_EXTRACT_SCALAR(Event_Value,"$.cross_sell") AS cross_sell
      , JSON_EXTRACT_SCALAR(Event_Value,"$.af_content_type") AS product
    FROM `youse-bigquery-dw.tests.appsflyer_test1`
  ),

  google_analytics_transactions AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , DATETIME(TIMESTAMP_MILLIS(visitStartTime*1000 + hits.time), "America/Sao_Paulo") AS conversion_time
      , CONCAT(trafficSource.source, " / ", trafficSource.medium) AS source_medium
      , (hits.transaction.transactionRevenue / 1e6) AS revenue
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 16) AS cross_sell
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 1) AS sales_channel
      , hits.transaction.transactionId AS transaction_id
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "TRANSACTION"
  ),

  google_analytics_items AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , hits.transaction.transactionId AS transaction_id
      , hits.item.productName AS product
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "ITEM"
  ),

  google_analytics AS (
    SELECT
      ga_transactions.hashed_email
      , ga_transactions.conversion_time
      , ga_transactions.source_medium
      , ga_transactions.revenue
      , ga_transactions.cross_sell
      , ga_items.product
      , ga_transactions.sales_channel
      , ga_transactions.transaction_id
    FROM
      google_analytics_transactions ga_transactions
    LEFT JOIN
      google_analytics_items ga_items
    ON 
      ga_transactions.hashed_email = ga_items.hashed_email
      AND ga_transactions.transaction_id = ga_items.transaction_id
  ),

  appsflyer_google_analytics AS (
    SELECT 
      af.hashed_email AS af_hashed_email
      , af.conversion_time AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , af.source_medium AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , af.revenue AS af_revenue
      , ga.revenue AS ga_revenue
      , af.cross_sell AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , af.product AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
      , ROW_NUMBER() OVER (
          PARTITION BY 
            af.hashed_email
            , af.conversion_time 
          ORDER BY 
            ABS(DATETIME_DIFF(ga.conversion_time, DATETIME(af.conversion_time),SECOND))) AS row_number
    FROM
      appsflyer af
    LEFT JOIN google_analytics ga
    ON 
      af.hashed_email = ga.hashed_email
      AND af.revenue = ga.revenue
    ORDER BY
      af.hashed_email 
  ),

  appsflyer_deduplicated AS (
    SELECT
      af_hashed_email
      , af_conversion_time
      , ga_conversion_time
      , af_source_medium
      , ga_source_medium
      , af_revenue
      , ga_revenue
      , af_cross_sell
      , ga_cross_sell 
      , ga_transaction_id
      , af_product
      , ga_product
      , ga_sales_channel
    FROM
      appsflyer_google_analytics
    WHERE 
      row_number = 1
      --removing rows without GA Transaction ID, that is, transactions that don't match GA
      AND ga_transaction_id IS NOT NULL
  )

  SELECT
    * 
  FROM
    appsflyer_deduplicated
  LIMIT 10

,hashed_email,af_conversion_time,ga_conversion_time,af_source_medium,ga_source_medium,af_revenue,ga_revenue,af_cross_sell,ga_cross_sell,sales_channel,transaction_id,product,row_number
0,0051600b756a99f40b451aced72c9290998d4750cdd3a9...,2019-09-26 21:30:05+00:00,2019-09-26 21:32:08.046,app / muvmobile_int,google / cpc,140.14,140.14,false,false,Digital,5003110344856,Seguro Auto,1
1,00677ba7557c556d4cd015dc064329e631c4bcfc0b4e03...,2019-09-06 23:03:53+00:00,2019-09-06 23:03:56.639,app / googleadwords_int,app / app_youse,179.38,179.38,false,false,Digital,5003110337342,Seguro Auto,1
2,00e37cb4c615ca912d547279407f4cab0c640d0de442c5...,2019-09-11 20:15:58+00:00,2019-09-11 20:16:03.122,app / doubleclick_int,app / app_youse,169.31,169.31,false,false,Digital,5003110338961,Seguro Auto,1
3,0120036244c1d5f9ccb335798e9fd275cc36798aa6bed4...,2019-09-25 17:27:53+00:00,2019-09-25 17:27:58.110,app / googleadwords_int,202616.apps.zdusercontent.com / referral,10.63,10.63,false,false,Call-Center,5001410102479,Seguro Residencial,1
4,01260d5174081fe48a54c63c18ce21ebddbcbcce57c107...,2019-09-16 16:19:10+00:00,2019-09-16 16:19:13.977,app / googleadwords_int,app / app_youse,192.05,192.05,false,false,Digital,5003110340802,Seguro Auto,1
5,0126a19cc233dc07789cce733cc8a6b8ae2ef452d5eb8e...,2019-09-17 15:19:06+00:00,2019-09-17 15:19:11.323,app / googleadwords_int,app / app_youse,69.92,69.92,false,false,Digital,5009110073789,Seguro Vida,1
6,012805e83c2233addde70c94d6b0b7cdd0d95906eb5aed...,2019-09-05 20:16:09+00:00,2019-09-05 20:16:12.177,app / googleadwords_int,202616.apps.zdusercontent.com / referral,226.81,226.81,false,false,Call-Center,5003110336827,Seguro Auto,1
7,018ae3d537666068939c65f3c57e7d87e2ba09d0726d41...,2019-09-27 19:13:10+00:00,2019-09-27 19:13:15.403,app / googleadwords_int,202616.apps.zdusercontent.com / referral,100.21,100.21,false,false,Call-Center,5003110345258,Seguro Auto,1
8,02375af5544a31171a57aebdf2e8d5a9cfac91727609ad...,2019-09-12 09:45:53+00:00,2019-09-12 09:45:59.205,app / googleadwords_int,google / cpc,54.02,54.02,false,false,Digital,5003110339130,Seguro Auto,1
9,024bdc8c255efe67c3b0b3064c63dd2945bc9664a5fdf9...,2019-09-16 10:19:52+00:00,2019-09-16 10:21:56.553,app / googleadwords_int,(direct) / (none),52.84,52.84,false,false,Digital,5003110340606,Seguro Auto,1


**Querying data from GA without Appsflyer**

In [0]:
%%bigquery --project youse-bigquery-dw
WITH 
  appsflyer AS (
    SELECT
      Customer_User_Id AS hashed_email
      , Event_Time AS conversion_time
      , CONCAT ("app / ", Media_Source) AS source_medium
      , Event_Revenue AS revenue
      , JSON_EXTRACT_SCALAR(Event_Value,"$.cross_sell") AS cross_sell
      , JSON_EXTRACT_SCALAR(Event_Value,"$.af_content_type") AS product
    FROM `youse-bigquery-dw.tests.appsflyer_test1`
  ),

  google_analytics_transactions AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , DATETIME(TIMESTAMP_MILLIS(visitStartTime*1000 + hits.time), "America/Sao_Paulo") AS conversion_time
      , CONCAT(trafficSource.source, " / ", trafficSource.medium) AS source_medium
      , (hits.transaction.transactionRevenue / 1e6) AS revenue
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 16) AS cross_sell
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 1) AS sales_channel
      , hits.transaction.transactionId AS transaction_id
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "TRANSACTION"
  ),

  google_analytics_items AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , hits.transaction.transactionId AS transaction_id
      , hits.item.productName AS product
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "ITEM"
  ),

  google_analytics AS (
    SELECT
      ga_transactions.hashed_email
      , ga_transactions.conversion_time
      , ga_transactions.source_medium
      , ga_transactions.revenue
      , ga_transactions.cross_sell
      , ga_items.product
      , ga_transactions.sales_channel
      , ga_transactions.transaction_id
    FROM
      google_analytics_transactions ga_transactions
    LEFT JOIN
      google_analytics_items ga_items
    ON 
      ga_transactions.hashed_email = ga_items.hashed_email
      AND ga_transactions.transaction_id = ga_items.transaction_id
  ),

  appsflyer_google_analytics AS (
    SELECT 
      af.hashed_email AS af_hashed_email
      , af.conversion_time AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , af.source_medium AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , af.revenue AS af_revenue
      , ga.revenue AS ga_revenue
      , af.cross_sell AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , af.product AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
      , ROW_NUMBER() OVER (
          PARTITION BY 
            af.hashed_email
            , af.conversion_time 
          ORDER BY 
            ABS(DATETIME_DIFF(ga.conversion_time, DATETIME(af.conversion_time),SECOND))) AS row_number
    FROM
      appsflyer af
    LEFT JOIN google_analytics ga
    ON 
      af.hashed_email = ga.hashed_email
      AND af.revenue = ga.revenue
    ORDER BY
      af.hashed_email 
  ),

  appsflyer_deduplicated AS (
    SELECT
      af_hashed_email
      , af_conversion_time
      , ga_conversion_time
      , af_source_medium
      , ga_source_medium
      , af_revenue
      , ga_revenue
      , af_cross_sell
      , ga_cross_sell 
      , ga_transaction_id
      , af_product
      , ga_product
      , ga_sales_channel
    FROM
      appsflyer_google_analytics
    WHERE 
      row_number = 1
      --removing rows without GA Transaction ID, that is, transactions that don't match GA
      AND ga_transaction_id IS NOT NULL
  ),
  
  google_analytics_without_appsflyer AS (
    SELECT
      hashed_email AS af_hashed_email
      , TIMESTAMP_TRUNC(CURRENT_TIMESTAMP, YEAR, "America/Sao_Paulo") AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , "N/A" AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , 0 AS af_revenue
      , ga.revenue AS ga_revenue
      , "N/A" AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , "N/A" AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
    FROM
      google_analytics ga
    LEFT JOIN appsflyer_deduplicated afd
    ON 
      ga.transaction_id = afd.ga_transaction_id
    WHERE
      afd.ga_transaction_id IS NULL
  )

SELECT
  *
FROM 
  google_analytics_without_appsflyer

**Putting all the data together again and setting just one source_medium we get the data we wanted**

In [0]:
%%bigquery --project youse-bigquery-dw
WITH 
  appsflyer AS (
    SELECT
      Customer_User_Id AS hashed_email
      , Event_Time AS conversion_time
      , CONCAT ("app / ", Media_Source) AS source_medium
      , Event_Revenue AS revenue
      , JSON_EXTRACT_SCALAR(Event_Value,"$.cross_sell") AS cross_sell
      , JSON_EXTRACT_SCALAR(Event_Value,"$.af_content_type") AS product
    FROM `youse-bigquery-dw.tests.appsflyer_test1`
  ),

  google_analytics_transactions AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , DATETIME(TIMESTAMP_MILLIS(visitStartTime*1000 + hits.time), "America/Sao_Paulo") AS conversion_time
      , CONCAT(trafficSource.source, " / ", trafficSource.medium) AS source_medium
      , (hits.transaction.transactionRevenue / 1e6) AS revenue
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 16) AS cross_sell
      , (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 1) AS sales_channel
      , hits.transaction.transactionId AS transaction_id
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "TRANSACTION"
  ),

  google_analytics_items AS (
    SELECT
      (SELECT value FROM UNNEST(hits.customDimensions) WHERE INDEX = 2) AS hashed_email
      , hits.transaction.transactionId AS transaction_id
      , hits.item.productName AS product
    FROM
      `youse-bigquery-dw.120767798.ga_sessions_*`,
      UNNEST(hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN "20190901" AND "20190930"
      AND hits.type = "ITEM"
  ),

  google_analytics AS (
    SELECT
      ga_transactions.hashed_email
      , ga_transactions.conversion_time
      , ga_transactions.source_medium
      , ga_transactions.revenue
      , ga_transactions.cross_sell
      , ga_items.product
      , ga_transactions.sales_channel
      , ga_transactions.transaction_id
    FROM
      google_analytics_transactions ga_transactions
    LEFT JOIN
      google_analytics_items ga_items
    ON 
      ga_transactions.hashed_email = ga_items.hashed_email
      AND ga_transactions.transaction_id = ga_items.transaction_id
  ),

  appsflyer_google_analytics AS (
    SELECT 
      af.hashed_email AS af_hashed_email
      , af.conversion_time AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , af.source_medium AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , af.revenue AS af_revenue
      , ga.revenue AS ga_revenue
      , af.cross_sell AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , af.product AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
      , ROW_NUMBER() OVER (
          PARTITION BY 
            af.hashed_email
            , af.conversion_time 
          ORDER BY 
            ABS(DATETIME_DIFF(ga.conversion_time, DATETIME(af.conversion_time),SECOND))) AS row_number
    FROM
      appsflyer af
    LEFT JOIN google_analytics ga
    ON 
      af.hashed_email = ga.hashed_email
      AND af.revenue = ga.revenue
    ORDER BY
      af.hashed_email 
  ),

  appsflyer_deduplicated AS (
    SELECT
      af_hashed_email
      , af_conversion_time
      , ga_conversion_time
      , af_source_medium
      , ga_source_medium
      , af_revenue
      , ga_revenue
      , af_cross_sell
      , ga_cross_sell 
      , ga_transaction_id
      , af_product
      , ga_product
      , ga_sales_channel
    FROM
      appsflyer_google_analytics
    WHERE 
      row_number = 1
      --removing rows without GA Transaction ID, that is, transactions that don't match GA
      AND ga_transaction_id IS NOT NULL
  ),
  
  google_analytics_without_appsflyer AS (
    SELECT
      hashed_email AS af_hashed_email
      , TIMESTAMP_TRUNC(CURRENT_TIMESTAMP, YEAR, "America/Sao_Paulo") AS af_conversion_time
      , ga.conversion_time AS ga_conversion_time
      , "N/A" AS af_source_medium
      , ga.source_medium AS ga_source_medium
      , 0 AS af_revenue
      , ga.revenue AS ga_revenue
      , "N/A" AS af_cross_sell
      , ga.cross_sell AS ga_cross_sell 
      , ga.transaction_id AS ga_transaction_id
      , "N/A" AS af_product
      , ga.product AS ga_product
      , ga.sales_channel AS ga_sales_channel
    FROM
      google_analytics ga
    LEFT JOIN appsflyer_deduplicated afd
    ON 
      ga.transaction_id = afd.ga_transaction_id
    WHERE
      afd.ga_transaction_id IS NULL
  ),
  
  appsflyer_google_analytics_full AS (
    SELECT * FROM appsflyer_deduplicated 
    UNION ALL
    SELECT * FROM google_analytics_without_appsflyer
  ),
  
  appsflyer_google_analytics_full_unique_source AS (
    SELECT
      af_hashed_email
      , af_conversion_time
      , ga_conversion_time
      , IF(af_source_medium = "N/A", ga_source_medium, af_source_medium) AS source_medium
      , af_revenue
      , ga_revenue
      , af_cross_sell
      , ga_cross_sell 
      , ga_transaction_id
      , af_product
      , ga_product
      , ga_sales_channel
    FROM 
      appsflyer_google_analytics_full
  )

SELECT 
  * 
FROM 
  appsflyer_google_analytics_full_unique_source
WHERE
  --for Call Center transactions set "Call-Center"
  ga_sales_channel = "Digital"